In [69]:
import nltk
from nltk.book import *
import numpy as np
import requests
from nltk.corpus import gutenberg
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('gutenberg')
nltk.download('punkt')

[nltk_data] Downloading package gutenberg to /Users/pasha/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /Users/pasha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Task 1.1
* *Implement Wagner-Fischer (or Vintsyuk algorithm) for string distance.*

In [2]:
def wagner_fischer_distance(string_1, string_2):
    string_1, string_2 = string_1.lower(), string_2.lower()
    
    d = [[0] * (len(string_2) + 1) for _ in range(len(string_1) + 1)]
    
    for i in range(1, len(string_1) + 1):
        d[i][0] = i
    for j in range(1, len(string_2) + 1):
        d[0][j] = j
    
    for j in range(1, len(string_2)+1):
        for i in range(1, len(string_1) + 1):
            if string_1[i - 1] == string_2[j - 1]:
                substitution_cost = 0
            else:
                substitution_cost = 1
            
            d[i][j] = min(
                d[i - 1][j] + 1,
                d[i][j - 1] + 1, 
                d[i - 1][j - 1] + substitution_cost
            )

    return d[len(string_1)][len(string_2)]

In [3]:
wagner_fischer_distance("Saturday", "Sunday")

3

## Task 1.2 
* *Modify the algorithm so that substitution operation cost depends on the key proximity on QWERTY keyboard.*

In [4]:
def get_github_file(url='https://gist.githubusercontent.com/pxeger/ae20549834c04f1e40a4fa4ba91a0079/raw/3cda3758aa5fae76dfdd574d042e0db8515d628d/keys.txt'):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Помилка при завантаженні файлу. Статус код: {response.status_code}")
        
def get_qwerty_response(qwert_respose_string):
    qwerty_response_ = {}
    
    for elmnt in qwert_respose_string.lower().split("\n"):
        if elmnt[0] in qwerty_response_:
            qwerty_response_[f"{elmnt[0]}"][str(elmnt.split()[1])] = float(elmnt.split()[-1])
        else :
            qwerty_response_[f"{elmnt[0]}"] = {str(elmnt.split()[1]): float(elmnt.split()[-1])}
    return qwerty_response_

In [5]:
qwerty_response = get_qwerty_response(get_github_file())
qwerty_response

{'a': {'a': 0.0,
  'b': 87.82,
  'c': 51.29,
  'd': 38.1,
  'e': 38.4,
  'f': 57.15,
  'g': 76.2,
  'h': 95.25,
  'i': 129.99,
  'j': 114.3,
  'k': 133.35,
  'l': 152.4,
  'm': 125.28,
  'n': 106.49,
  'o': 148.86,
  'p': 167.77,
  'q': 19.64,
  'r': 55.74,
  's': 19.05,
  't': 73.93,
  'u': 111.18,
  'v': 69.34,
  'w': 23.81,
  'x': 34.34,
  'y': 92.47,
  'z': 21.3},
 'b': {'a': 87.82,
  'b': 0.0,
  'c': 38.1,
  'd': 51.29,
  'e': 64.78,
  'f': 34.34,
  'g': 21.3,
  'h': 21.3,
  'i': 57.35,
  'j': 34.34,
  'k': 51.29,
  'l': 69.34,
  'm': 38.1,
  'n': 19.05,
  'o': 72.7,
  'p': 89.48,
  'q': 98.18,
  'r': 50.63,
  's': 69.34,
  't': 40.69,
  'u': 44.93,
  'v': 19.05,
  'w': 80.96,
  'x': 57.15,
  'y': 38.4,
  'z': 76.2},
 'c': {'a': 51.29,
  'b': 38.1,
  'c': 0.0,
  'd': 21.3,
  'e': 40.69,
  'f': 21.3,
  'g': 34.34,
  'h': 51.29,
  'i': 89.48,
  'j': 69.34,
  'k': 87.82,
  'l': 106.49,
  'm': 76.2,
  'n': 57.15,
  'o': 107.02,
  'p': 125.01,
  'q': 64.78,
  'r': 38.4,
  's': 34.34,
 

In [6]:
def qwerty_distance(string_1, string_2,*, qwerty_response=qwerty_response):
    string_1, string_2 = string_1.lower(), string_2.lower()
    
    d = [[0] * (len(string_2) + 1) for _ in range(len(string_1) + 1)]
    
    for i in range(1, len(string_1) + 1):
        d[i][0] = i
    for j in range(1, len(string_2) + 1):
        d[0][j] = j
    
    for j in range(1, len(string_2)+1):
        for i in range(1, len(string_1) + 1):
            
            substitution_cost = qwerty_response[string_1[i - 1]][string_2[j - 1]]
            
            d[i][j] = min(
                d[i - 1][j] + 1,
                d[i][j - 1] + 1, 
                d[i - 1][j - 1] + substitution_cost
            )

    return d[len(string_1)][len(string_2)]

In [7]:
qwerty_distance("Saturday", "Sunday")

4.0

## Task 1.3 
* *Include transposition operation, so that you compute a Damerau-Levenshtein distance*

In [8]:
def damerau_levenshtein_distance(string_1, string_2):
    string_1, string_2 = string_1.lower(), string_2.lower()
    
    d = [[0] * (len(string_2) + 1) for _ in range(len(string_1) + 1)]
    
    for i in range(1, len(string_1) + 1):
        d[i][0] = i
    for j in range(1, len(string_2) + 1):
        d[0][j] = j
    
    for i in range(1, len(string_1) + 1):
        for j in range(1, len(string_2) + 1):
            if string_1[i-1] == string_2[j-1]:
                substitution_cost = 0
            else:
                substitution_cost = 1
            
            d[i][j] = min(
                d[i - 1][j] + 1,
                d[i][j - 1] + 1, 
                d[i - 1][j - 1] + substitution_cost
            )
            
            if i > 1 and j > 1 and string_1[i-1] == string_2[j-2] and string_1[i-2] == string_2[j-1]:
                d[i][j] = min(
                    d[i][j],
                    d[i-2][j-2]+substitution_cost
                )

    return d[len(string_1)][len(string_2)]

In [9]:
damerau_levenshtein_distance("Saturday", "Sunday")

3

## Task 2 
* *Take an arbitrary text from NLTK corpora (e.g. text3) and implement a Bag-of-Words tagger for it* 
 


In [71]:
def bag_of_words(text=gutenberg.raw('melville-moby_dick.txt')):
    my_dict = {element: 0 for element in set([word.lower() for word in word_tokenize(moby_dick_raw) if word.isalpha()])}

    for word in [word.lower() for word in word_tokenize(moby_dick_raw) if word.isalpha()]:
        if word in my_dict:
            my_dict[word] = my_dict[word]+1
    return list(my_dict.values())

In [72]:
bag_of_words()[:15]

[2, 1, 7, 15, 1, 2, 3, 1, 1, 11, 3, 2, 2, 2, 1]

## Task 2.1 
* *Create a vocabulary of all words in a given corpus: ti, i ∈ [1,N], where N is the size of the vocabulary*

In [42]:
def vocabulary_of_text(text=gutenberg.raw('melville-moby_dick.txt')):
    return sorted(set([word.lower() for word in word_tokenize(moby_dick_raw) if word.isalpha()]))

In [46]:
moby_dick_voc = vocabulary_of_text()
moby_dick_voc[:15]

['a',
 'aback',
 'abaft',
 'abandon',
 'abandoned',
 'abandonedly',
 'abandonment',
 'abased',
 'abasement',
 'abashed',
 'abate',
 'abated',
 'abatement',
 'abating',
 'abbreviate']

## Task 2.2
* *Given a document d, calculate number of times each word ti occurs and denote it count(ti, d)*

In [51]:
def count_occurs(text_voc=moby_dick_voc, text=gutenberg.raw('melville-moby_dick.txt')):
    
    my_dict = {element: 0 for element in text_voc}

    for word in [word.lower() for word in word_tokenize(moby_dick_raw) if word.isalpha()]:
        if word in my_dict:
            my_dict[word] = my_dict[word]+1
    return my_dict

In [53]:
moby_dick_words_occurs = count_occurs()
moby_dick_words_occurs

{'a': 4698,
 'aback': 2,
 'abaft': 2,
 'abandon': 3,
 'abandoned': 7,
 'abandonedly': 1,
 'abandonment': 2,
 'abased': 2,
 'abasement': 1,
 'abashed': 2,
 'abate': 1,
 'abated': 3,
 'abatement': 1,
 'abating': 2,
 'abbreviate': 1,
 'abbreviation': 1,
 'abeam': 1,
 'abed': 2,
 'abednego': 1,
 'abel': 1,
 'abhorred': 3,
 'abhorrence': 1,
 'abhorrent': 1,
 'abhorring': 1,
 'abide': 3,
 'abided': 1,
 'abiding': 1,
 'ability': 1,
 'abjectly': 1,
 'abjectus': 1,
 'able': 9,
 'ablutions': 2,
 'aboard': 21,
 'abode': 2,
 'abominable': 3,
 'abominate': 1,
 'abominated': 1,
 'abomination': 1,
 'aboriginal': 4,
 'aboriginally': 1,
 'aboriginalness': 1,
 'abortion': 1,
 'abortions': 1,
 'abound': 3,
 'abounded': 1,
 'abounding': 7,
 'aboundingly': 1,
 'about': 310,
 'above': 56,
 'abraham': 3,
 'abreast': 3,
 'abridged': 2,
 'abroad': 6,
 'abruptly': 2,
 'absence': 5,
 'absent': 6,
 'absolute': 4,
 'absolutely': 3,
 'absorbed': 4,
 'absorbing': 2,
 'absorbingly': 1,
 'abstained': 1,
 'abstemious':

## Task 2.3
* *Map this document to an N-dimensional vector, where on each i-th position we’ll have count(ti, d)*

In [58]:
def map_to_vector(words_occurs=moby_dick_words_occurs):
    return list(words_occurs.values())

In [62]:
moby_dick_vector = map_to_vector()
moby_dick_vector[:15]

[4698, 2, 2, 3, 7, 1, 2, 2, 1, 2, 1, 3, 1, 2, 1]

## Task 2.4
* *Compute distances between two documents via dot product*

In [89]:
text_1 = gutenberg.raw('melville-moby_dick.txt')
text_2 = gutenberg.raw('shakespeare-macbeth.txt')

In [90]:
bag_of_words_text_1 = np.array(bag_of_words(text_1))
bag_of_words_text_2 = np.array(bag_of_words(text_2))

In [91]:
normalized_vector_text_1 = bag_of_words_text_1 / np.linalg.norm(bag_of_words_text_1)
normalized_vector_text_2 = bag_of_words_text_2 / np.linalg.norm(bag_of_words_text_2)

In [92]:
print(f"Добуток (косинусна схожість) між документами: {np.dot(normalized_vector_text_1, normalized_vector_text_2)}")

Добуток (косинусна схожість) між документами: 1.0000000000000089
